In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
print('DEVICES AVAILABLE: {}'.format(strategy.num_replicas_in_sync))

DEVICES AVAILABLE: 2


# Comparing V2L and V2S

In [1]:
import tensorflow.keras.applications as pmodels
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import image_dataset_from_directory
import tensorflow as tf
strategy = tf.distribute.MirroredStrategy()

def test_base_models():
    images = image_dataset_from_directory("/kaggle/input/stanford-dogs-dataset-traintest/cropped/train", 
                                      seed=1234, subset="both", validation_split=0.2, batch_size=32, 
                                      labels="inferred", label_mode = "categorical")
    test_images = image_dataset_from_directory("/kaggle/input/stanford-dogs-dataset-traintest/cropped/test", 
                                      seed=1234, batch_size=32, 
                                      labels="inferred", label_mode = "categorical")
    base_models = [pmodels.efficientnet_v2.EfficientNetV2L, pmodels.efficientnet_v2.EfficientNetV2S]
    preprocess_func = [do_nothing, do_nothing]
    histories = []
    scores = []
    models = []
    for index, base_model in enumerate(base_models):
        model = build_model(base_model)
        images_train = images[0].map(lambda x, y: (preprocess_func[index](x), y))
        images_val = images[1].map(lambda x, y: (preprocess_func[index](x), y))
        images_test = test_images.map(lambda x, y: (preprocess_func[index](x), y))
        history = train_model(model, images_train, images_val)
        score = evaluate_model(model, images_test)
        histories.append(history)
        scores.append(score)
        models.append(model)
    return histories, scores, models, base_models

def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    flattening_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(120, activation='softmax')

    model = models.Sequential([
      model,
      flattening_layer,
      dense_layer,
      prediction_layer
    ])

    return model

def build_model(base_model):
    with strategy.scope():
        model = load_model(base_model)
        model = set_nontrainable_layers(model)
        model = add_last_layers(model)
        model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(),
                  metrics=['accuracy'])
        return model

def load_model(base_model):
    model = base_model(include_top=False, weights='imagenet', input_shape=(256, 256, 3))
    return model

def set_nontrainable_layers(model):
    model.trainable = False
    return model

def train_model(model, images_train, images_val):
    model.summary()
    es = EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True)
    history = model.fit(images_train,
              epochs=100,
              batch_size=32,
              callbacks=es,
              validation_data=images_val,
              verbose=1)
    return history

def evaluate_model(model, images_test):
    return model.evaluate(images_test)

def do_nothing(object):
    return object

histories, scores, models, base_models = test_base_models()

2024-02-08 11:12:48.317280: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-08 11:12:48.317374: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-08 11:12:48.625325: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 12000 files belonging to 120 classes.
Using 9600 files for training.
Using 2400 files for validation.
Found 8580 files belonging to 120 classes.
473176280/473176280 [==============================] - 4s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-l (Function  (None, 8, 8, 1280)        117746848 
 al)                                                             
                                                                 
 flatten (Flatten)           (None, 81920)             0         
                                                                 
 dense (Dense)               (None, 500)               40960500  
                                                                 
 dense_1 (Dense)             (None, 120)               60120     
                                                                 
Total params: 158767468 (605.65 MB)


2024-02-08 11:14:36.790426: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/efficientnetv2-l/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1707390886.446468     103 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


300/300 [==============================] - 203s 450ms/step - loss: 2.2275 - accuracy: 0.7496 - val_loss: 0.5395 - val_accuracy: 0.8763
Epoch 2/100
300/300 [==============================] - 118s 394ms/step - loss: 0.5879 - accuracy: 0.8930 - val_loss: 0.6440 - val_accuracy: 0.8637
Epoch 3/100
300/300 [==============================] - 119s 396ms/step - loss: 0.3857 - accuracy: 0.9170 - val_loss: 0.6705 - val_accuracy: 0.8842
Epoch 4/100
300/300 [==============================] - 119s 397ms/step - loss: 0.3372 - accuracy: 0.9248 - val_loss: 0.7198 - val_accuracy: 0.8792
Epoch 5/100
300/300 [==============================] - 118s 393ms/step - loss: 0.3317 - accuracy: 0.9274 - val_loss: 0.8728 - val_accuracy: 0.8717
Epoch 6/100
82420632/82420632 [==============================] - 0s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-s (Function  (None, 8, 8, 1280

2024-02-08 11:28:52.177324: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_1/efficientnetv2-s/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


300/300 [==============================] - 97s 220ms/step - loss: 1.6200 - accuracy: 0.7352 - val_loss: 0.8083 - val_accuracy: 0.8408
Epoch 2/100
300/300 [==============================] - 56s 185ms/step - loss: 0.4382 - accuracy: 0.9041 - val_loss: 0.9845 - val_accuracy: 0.8438
Epoch 3/100
300/300 [==============================] - 56s 187ms/step - loss: 0.4621 - accuracy: 0.9209 - val_loss: 0.8150 - val_accuracy: 0.8717
Epoch 4/100
300/300 [==============================] - 55s 182ms/step - loss: 0.3464 - accuracy: 0.9353 - val_loss: 1.0838 - val_accuracy: 0.8558
Epoch 5/100
300/300 [==============================] - 55s 183ms/step - loss: 0.2851 - accuracy: 0.9461 - val_loss: 1.0363 - val_accuracy: 0.8637
Epoch 6/100
269/269 [==============================] - 30s 108ms/step - loss: 0.7186 - accuracy: 0.8871


.89 vs. .8871 performance on the test set makes not much of a difference, but runtime is halfed --> using V2S for now.

# Trying out data augmentation

In [4]:
import tensorflow.keras.applications as pmodels
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import image_dataset_from_directory
import tensorflow as tf
strategy = tf.distribute.MirroredStrategy()

def test_base_models():
    images = image_dataset_from_directory("/kaggle/input/stanford-dogs-dataset-traintest/cropped/train", 
                                      seed=1234, subset="both", validation_split=0.2, batch_size=32, 
                                      labels="inferred", label_mode = "categorical")
    test_images = image_dataset_from_directory("/kaggle/input/stanford-dogs-dataset-traintest/cropped/test", 
                                      seed=1234, batch_size=32, 
                                      labels="inferred", label_mode = "categorical")
    base_models = [pmodels.efficientnet_v2.EfficientNetV2S]
    preprocess_func = [do_nothing]
    histories = []
    scores = []
    models = []
    for index, base_model in enumerate(base_models):
        model = build_model(base_model)
        images_train = images[0].map(lambda x, y: (preprocess_func[index](x), y))
        data_augmentation = tf.keras.Sequential([layers.RandomFlip("horizontal")])
        images_train = images_train.map(lambda x, y: (data_augmentation(x, training=True), y))
        images_val = images[1].map(lambda x, y: (preprocess_func[index](x), y))
        images_test = test_images.map(lambda x, y: (preprocess_func[index](x), y))
        history = train_model(model, images_train, images_val)
        score = evaluate_model(model, images_test)
        histories.append(history)
        scores.append(score)
        models.append(model)
    return histories, scores, models, base_models

def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    flattening_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(120, activation='softmax')

    model = models.Sequential([
      model,
      flattening_layer,
      dense_layer,
      prediction_layer
    ])

    return model

def build_model(base_model):
    with strategy.scope():
        model = load_model(base_model)
        model = set_nontrainable_layers(model)
        model = add_last_layers(model)
        model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(),
                  metrics=['accuracy'])
        return model

def load_model(base_model):
    model = base_model(include_top=False, weights='imagenet', input_shape=(256, 256, 3))
    return model

def set_nontrainable_layers(model):
    model.trainable = False
    return model

def train_model(model, images_train, images_val):
    model.summary()
    es = EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True)
    history = model.fit(images_train,
              epochs=100,
              batch_size=32,
              callbacks=es,
              validation_data=images_val,
              verbose=1)
    return history

def evaluate_model(model, images_test):
    return model.evaluate(images_test)

def do_nothing(object):
    return object

histories, scores, models, base_models = test_base_models()

Found 12000 files belonging to 120 classes.
Using 9600 files for training.
Using 2400 files for validation.
Found 8580 files belonging to 120 classes.
Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-s (Function  (None, 8, 8, 1280)        20331360  
 al)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 81920)             0         
                                                                 
 dense_8 (Dense)             (None, 500)               40960500  
                                                                 
 dense_9 (Dense)             (None, 120)               60120     
                                                                 
Total params: 61351980 (234.04 MB)
Trainable params: 41020620 (156.48 MB)
Non-trainable params: 20331

2024-02-08 12:00:06.684793: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_6/efficientnetv2-s/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


300/300 [==============================] - 95s 209ms/step - loss: 1.5298 - accuracy: 0.7371 - val_loss: 0.6731 - val_accuracy: 0.8517
Epoch 2/100
300/300 [==============================] - 56s 186ms/step - loss: 0.5707 - accuracy: 0.8734 - val_loss: 0.7294 - val_accuracy: 0.8725
Epoch 3/100
300/300 [==============================] - 55s 183ms/step - loss: 0.4725 - accuracy: 0.9033 - val_loss: 0.8077 - val_accuracy: 0.8604
Epoch 4/100
300/300 [==============================] - 55s 184ms/step - loss: 0.4958 - accuracy: 0.9065 - val_loss: 0.9524 - val_accuracy: 0.8583
Epoch 5/100
269/269 [==============================] - 25s 91ms/step - loss: 0.6784 - accuracy: 0.8724


* 0.884 with horizontal flipping and random rotation
* 0.8724 with just horizontal flipping

# Changing batch size

In [6]:
import tensorflow.keras.applications as pmodels
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import image_dataset_from_directory
import tensorflow as tf
strategy = tf.distribute.MirroredStrategy()

def test_base_models():
    images = image_dataset_from_directory("/kaggle/input/stanford-dogs-dataset-traintest/cropped/train", 
                                      seed=1234, subset="both", validation_split=0.2, batch_size=64, 
                                      labels="inferred", label_mode = "categorical")
    test_images = image_dataset_from_directory("/kaggle/input/stanford-dogs-dataset-traintest/cropped/test", 
                                      seed=1234, batch_size=32, 
                                      labels="inferred", label_mode = "categorical")
    base_models = [pmodels.efficientnet_v2.EfficientNetV2S]
    preprocess_func = [do_nothing]
    histories = []
    scores = []
    models = []
    for index, base_model in enumerate(base_models):
        model = build_model(base_model)
        images_train = images[0].map(lambda x, y: (preprocess_func[index](x), y))
        images_val = images[1].map(lambda x, y: (preprocess_func[index](x), y))
        images_test = test_images.map(lambda x, y: (preprocess_func[index](x), y))
        history = train_model(model, images_train, images_val)
        score = evaluate_model(model, images_test)
        histories.append(history)
        scores.append(score)
        models.append(model)
    return histories, scores, models, base_models

def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    flattening_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(120, activation='softmax')

    model = models.Sequential([
      model,
      flattening_layer,
      dense_layer,
      prediction_layer
    ])

    return model

def build_model(base_model):
    with strategy.scope():
        model = load_model(base_model)
        model = set_nontrainable_layers(model)
        model = add_last_layers(model)
        model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.Adam(),
                  metrics=['accuracy'])
        return model

def load_model(base_model):
    model = base_model(include_top=False, weights='imagenet', input_shape=(256, 256, 3))
    return model

def set_nontrainable_layers(model):
    model.trainable = False
    return model

def train_model(model, images_train, images_val):
    model.summary()
    es = EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True)
    history = model.fit(images_train,
              epochs=100,
              batch_size=64,
              callbacks=es,
              validation_data=images_val,
              verbose=1)
    return history

def evaluate_model(model, images_test):
    return model.evaluate(images_test)

def do_nothing(object):
    return object

histories, scores, models, base_models = test_base_models()

Found 12000 files belonging to 120 classes.
Using 9600 files for training.
Using 2400 files for validation.
Found 8580 files belonging to 120 classes.
Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-s (Function  (None, 8, 8, 1280)        20331360  
 al)                                                             
                                                                 
 flatten_6 (Flatten)         (None, 81920)             0         
                                                                 
 dense_12 (Dense)            (None, 500)               40960500  
                                                                 
 dense_13 (Dense)            (None, 120)               60120     
                                                                 
Total params: 61351980 (234.04 MB)
Trainable params: 41020620 (156.48 MB)
Non-trainable params: 20331

2024-02-08 12:11:36.481063: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_9/efficientnetv2-s/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


150/150 [==============================] - 89s 387ms/step - loss: 1.4773 - accuracy: 0.7542 - val_loss: 0.6679 - val_accuracy: 0.8504
Epoch 2/100
150/150 [==============================] - 46s 302ms/step - loss: 0.3825 - accuracy: 0.9167 - val_loss: 0.7771 - val_accuracy: 0.8650
Epoch 3/100
150/150 [==============================] - 46s 304ms/step - loss: 0.2131 - accuracy: 0.9444 - val_loss: 0.7995 - val_accuracy: 0.8725
Epoch 4/100
150/150 [==============================] - 46s 304ms/step - loss: 0.2026 - accuracy: 0.9505 - val_loss: 0.8744 - val_accuracy: 0.8817
Epoch 5/100
150/150 [==============================] - 46s 306ms/step - loss: 0.1790 - accuracy: 0.9633 - val_loss: 0.8837 - val_accuracy: 0.8821
Epoch 6/100
150/150 [==============================] - 47s 312ms/step - loss: 0.1696 - accuracy: 0.9660 - val_loss: 1.0070 - val_accuracy: 0.8796
Epoch 7/100
150/150 [==============================] - 45s 301ms/step - loss: 0.1600 - accuracy: 0.9641 - val_loss: 1.0660 - val_accurac

* worse for smaller batch size (16)
* better for bigger batch size (64): 0.8925

# Changing the output layer

Global Average Pooling instead of Dense Layer

In [17]:
import tensorflow.keras.applications as pmodels
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import image_dataset_from_directory
import tensorflow as tf
strategy = tf.distribute.MirroredStrategy()

def test_base_models():
    images = image_dataset_from_directory("/kaggle/input/stanford-dogs-dataset-traintest/cropped/train", 
                                      seed=1234, subset="both", validation_split=0.2, batch_size=64, 
                                      labels="inferred", label_mode = "categorical")
    test_images = image_dataset_from_directory("/kaggle/input/stanford-dogs-dataset-traintest/cropped/test", 
                                      seed=1234, batch_size=32, 
                                      labels="inferred", label_mode = "categorical")
    base_models = [pmodels.efficientnet_v2.EfficientNetV2S]
    preprocess_func = [do_nothing]
    histories = []
    scores = []
    models = []
    for index, base_model in enumerate(base_models):
        model = build_model(base_model)
        images_train = images[0].map(lambda x, y: (preprocess_func[index](x), y))
        images_val = images[1].map(lambda x, y: (preprocess_func[index](x), y))
        images_test = test_images.map(lambda x, y: (preprocess_func[index](x), y))
        history = train_model(model, images_train, images_val)
        score = evaluate_model(model, images_test)
        histories.append(history)
        scores.append(score)
        models.append(model)
    return histories, scores, models, base_models

def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    gap_layer = layers.GlobalAveragePooling2D()
    prediction_layer = layers.Dense(120, activation='softmax')

    model = models.Sequential([
      model,
      gap_layer,
      prediction_layer
    ])

    return model

def build_model(base_model):

    model = load_model(base_model)
    model = set_nontrainable_layers(model)
    model = add_last_layers(model)
    model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['accuracy'])
    return model

def load_model(base_model):
    model = base_model(include_top=False, weights='imagenet', input_shape=(256, 256, 3))
    return model

def set_nontrainable_layers(model):
    model.trainable = False
    return model

def train_model(model, images_train, images_val):
    model.summary()
    es = EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True)
    history = model.fit(images_train,
              epochs=100,
              batch_size=64,
              callbacks=es,
              validation_data=images_val,
              verbose=1)
    return history

def evaluate_model(model, images_test):
    return model.evaluate(images_test)

def do_nothing(object):
    return object

histories, scores, models, base_models = test_base_models()

Found 12000 files belonging to 120 classes.
Using 9600 files for training.
Using 2400 files for validation.
Found 8580 files belonging to 120 classes.
Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-s (Function  (None, 8, 8, 1280)        20331360  
 al)                                                             
                                                                 
 global_average_pooling2d_3  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_17 (Dense)            (None, 120)               153720    
                                                                 
Total params: 20485080 (78.14 MB)
Trainable params: 153720 (600.47 KB)
Non-trainable params: 20331360 (77.56 MB)
_____________________________________________________

2024-02-08 12:39:18.396919: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_13/efficientnetv2-s/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


150/150 [==============================] - 84s 448ms/step - loss: 1.4844 - accuracy: 0.8122 - val_loss: 0.3594 - val_accuracy: 0.9187
Epoch 2/100
150/150 [==============================] - 57s 378ms/step - loss: 0.3116 - accuracy: 0.9189 - val_loss: 0.2741 - val_accuracy: 0.9237
Epoch 3/100
150/150 [==============================] - 57s 377ms/step - loss: 0.2476 - accuracy: 0.9276 - val_loss: 0.2598 - val_accuracy: 0.9196
Epoch 4/100
150/150 [==============================] - 57s 376ms/step - loss: 0.2154 - accuracy: 0.9346 - val_loss: 0.2545 - val_accuracy: 0.9179
Epoch 5/100
269/269 [==============================] - 39s 145ms/step - loss: 0.2429 - accuracy: 0.9315


# GAP2D with Normalization and Dropout

In [18]:
import tensorflow.keras.applications as pmodels
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import image_dataset_from_directory
import tensorflow as tf
strategy = tf.distribute.MirroredStrategy()

def test_base_models():
    images = image_dataset_from_directory("/kaggle/input/stanford-dogs-dataset-traintest/cropped/train", 
                                      seed=1234, subset="both", validation_split=0.2, batch_size=64, 
                                      labels="inferred", label_mode = "categorical")
    test_images = image_dataset_from_directory("/kaggle/input/stanford-dogs-dataset-traintest/cropped/test", 
                                      seed=1234, batch_size=32, 
                                      labels="inferred", label_mode = "categorical")
    base_models = [pmodels.efficientnet_v2.EfficientNetV2S]
    preprocess_func = [do_nothing]
    histories = []
    scores = []
    models = []
    for index, base_model in enumerate(base_models):
        model = build_model(base_model)
        images_train = images[0].map(lambda x, y: (preprocess_func[index](x), y))
        images_val = images[1].map(lambda x, y: (preprocess_func[index](x), y))
        images_test = test_images.map(lambda x, y: (preprocess_func[index](x), y))
        history = train_model(model, images_train, images_val)
        score = evaluate_model(model, images_test)
        histories.append(history)
        scores.append(score)
        models.append(model)
    return histories, scores, models, base_models

def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    gap_layer = layers.GlobalAveragePooling2D()
    norm_layer = layers.BatchNormalization()
    dropout_layer = layers.Dropout(0.2)
    prediction_layer = layers.Dense(120, activation='softmax')

    model = models.Sequential([
      model,
      gap_layer,
      prediction_layer
    ])

    return model

def build_model(base_model):

    model = load_model(base_model)
    model = set_nontrainable_layers(model)
    model = add_last_layers(model)
    model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['accuracy'])
    return model

def load_model(base_model):
    model = base_model(include_top=False, weights='imagenet', input_shape=(256, 256, 3))
    return model

def set_nontrainable_layers(model):
    model.trainable = False
    return model

def train_model(model, images_train, images_val):
    model.summary()
    es = EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True)
    history = model.fit(images_train,
              epochs=100,
              batch_size=64,
              callbacks=es,
              validation_data=images_val,
              verbose=1)
    return history

def evaluate_model(model, images_test):
    return model.evaluate(images_test)

def do_nothing(object):
    return object

histories, scores, models, base_models = test_base_models()

Found 12000 files belonging to 120 classes.
Using 9600 files for training.
Using 2400 files for validation.
Found 8580 files belonging to 120 classes.
Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-s (Function  (None, 8, 8, 1280)        20331360  
 al)                                                             
                                                                 
 global_average_pooling2d_4  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_18 (Dense)            (None, 120)               153720    
                                                                 
Total params: 20485080 (78.14 MB)
Trainable params: 153720 (600.47 KB)
Non-trainable params: 20331360 (77.56 MB)
_____________________________________________________

2024-02-08 12:45:39.735562: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_14/efficientnetv2-s/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


150/150 [==============================] - 74s 402ms/step - loss: 1.4788 - accuracy: 0.8095 - val_loss: 0.3572 - val_accuracy: 0.9137
Epoch 2/100
150/150 [==============================] - 57s 377ms/step - loss: 0.3101 - accuracy: 0.9209 - val_loss: 0.2694 - val_accuracy: 0.9225
Epoch 3/100
150/150 [==============================] - 57s 376ms/step - loss: 0.2479 - accuracy: 0.9265 - val_loss: 0.2560 - val_accuracy: 0.9233
Epoch 4/100
150/150 [==============================] - 57s 377ms/step - loss: 0.2189 - accuracy: 0.9340 - val_loss: 0.2497 - val_accuracy: 0.9204
Epoch 5/100
150/150 [==============================] - 57s 377ms/step - loss: 0.1988 - accuracy: 0.9377 - val_loss: 0.2474 - val_accuracy: 0.9200
Epoch 6/100
269/269 [==============================] - 38s 140ms/step - loss: 0.2246 - accuracy: 0.9297


Adding Dense layers after global pooling

In [1]:
import tensorflow.keras.applications as pmodels
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import image_dataset_from_directory
import tensorflow as tf
strategy = tf.distribute.MirroredStrategy()

def test_base_models():
    images = image_dataset_from_directory("/kaggle/input/stanford-dogs-dataset-traintest/cropped/train", 
                                      seed=1234, subset="both", validation_split=0.2, batch_size=64, 
                                      labels="inferred", label_mode = "categorical")
    test_images = image_dataset_from_directory("/kaggle/input/stanford-dogs-dataset-traintest/cropped/test", 
                                      seed=1234, batch_size=32, 
                                      labels="inferred", label_mode = "categorical")
    base_models = [pmodels.efficientnet_v2.EfficientNetV2S]
    preprocess_func = [do_nothing]
    histories = []
    scores = []
    models = []
    for index, base_model in enumerate(base_models):
        model = build_model(base_model)
        images_train = images[0].map(lambda x, y: (preprocess_func[index](x), y))
        images_val = images[1].map(lambda x, y: (preprocess_func[index](x), y))
        images_test = test_images.map(lambda x, y: (preprocess_func[index](x), y))
        history = train_model(model, images_train, images_val)
        score = evaluate_model(model, images_test)
        histories.append(history)
        scores.append(score)
        models.append(model)
    return histories, scores, models, base_models

def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    gap_layer = layers.GlobalAveragePooling2D()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(120, activation='softmax')

    model = models.Sequential([
      model,
      gap_layer,
      dense_layer,
      prediction_layer
    ])

    return model

def build_model(base_model):

    model = load_model(base_model)
    model = set_nontrainable_layers(model)
    model = add_last_layers(model)
    model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['accuracy'])
    return model

def load_model(base_model):
    model = base_model(include_top=False, weights='imagenet', input_shape=(256, 256, 3))
    return model

def set_nontrainable_layers(model):
    model.trainable = False
    return model

def train_model(model, images_train, images_val):
    model.summary()
    es = EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True)
    history = model.fit(images_train,
              epochs=100,
              batch_size=64,
              callbacks=es,
              validation_data=images_val,
              verbose=1)
    return history

def evaluate_model(model, images_test):
    return model.evaluate(images_test)

def do_nothing(object):
    return object

histories, scores, models, base_models = test_base_models()

2024-02-08 13:00:43.157846: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-08 13:00:43.157946: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-08 13:00:43.292753: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 12000 files belonging to 120 classes.
Using 9600 files for training.
Using 2400 files for validation.
Found 8580 files belonging to 120 classes.
82420632/82420632 [==============================] - 3s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-s (Function  (None, 8, 8, 1280)        20331360  
 al)                                                             
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 500)               640500    
                                                                 
 dense_1 (Dense)             (None, 120)               60120     
                                      

2024-02-08 13:01:26.058254: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/efficientnetv2-s/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1707397292.653547      85 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


150/150 [==============================] - 84s 429ms/step - loss: 0.8492 - accuracy: 0.8314 - val_loss: 0.2887 - val_accuracy: 0.9112
Epoch 2/100
150/150 [==============================] - 58s 386ms/step - loss: 0.2594 - accuracy: 0.9190 - val_loss: 0.3038 - val_accuracy: 0.9021
Epoch 3/100
150/150 [==============================] - 58s 385ms/step - loss: 0.2081 - accuracy: 0.9299 - val_loss: 0.2957 - val_accuracy: 0.9100
Epoch 4/100
150/150 [==============================] - 58s 385ms/step - loss: 0.1804 - accuracy: 0.9397 - val_loss: 0.2969 - val_accuracy: 0.9146
Epoch 5/100
150/150 [==============================] - 58s 385ms/step - loss: 0.1562 - accuracy: 0.9461 - val_loss: 0.3060 - val_accuracy: 0.9062
Epoch 6/100
150/150 [==============================] - 58s 384ms/step - loss: 0.1293 - accuracy: 0.9559 - val_loss: 0.3164 - val_accuracy: 0.9087
Epoch 7/100
269/269 [==============================] - 40s 149ms/step - loss: 0.2612 - accuracy: 0.9245
